In [1]:
from requests import get
url = 'http://www.imdb.com/search/title?release_date=2017&sort=num_votes,desc&page=1'
response = get(url)
print(response.text[:500])



<!DOCTYPE html>
<html
    xmlns:og="http://ogp.me/ns#"
    xmlns:fb="http://www.facebook.com/2008/fbml">
    <head>
         
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">

    <meta name="apple-itunes-app" content="app-id=342792525, app-argument=imdb:///?src=mdot">



        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>

<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle",


In [2]:
from bs4 import BeautifulSoup
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

bs4.BeautifulSoup

In [3]:
movie_containers = html_soup.find_all('div', class_ = 'lister-item mode-advanced')  
print(type(movie_containers))
print(len(movie_containers))
first_movie = movie_containers[0]

<class 'bs4.element.ResultSet'>
50


In [4]:
print(first_movie)

<div class="lister-item mode-advanced">
<div class="lister-top-right">
<div class="ribbonize" data-caller="filmosearch" data-tconst="tt3315342"></div>
</div>
<div class="lister-item-image float-left">
<a href="/title/tt3315342/"> <img alt="Logan" class="loadlate" data-tconst="tt3315342" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BYzc5MTU4N2EtYTkyMi00NjdhLTg3NWEtMTY4OTEyMzJhZTAzXkEyXkFqcGdeQXVyNjc1NTYyMjg@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/G/01/imdb/images/nopicture/large/film-184890147._CB470041630_.png" width="67"/>
</a> </div>
<div class="lister-item-content">
<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt3315342/">Logan</a>
<span class="lister-item-year text-muted unbold">(2017)</span>
</h3>
<p class="text-muted">
<span class="certificate">R</span>
<span class="ghost">|</span>
<span class="runtime">137 min</span>
<span class="ghost">|</span>
<span class="genre">
Act

In [5]:
first_name = first_movie.h3.a.text
first_name

'Logan'

In [6]:
first_year = first_movie.h3.find('span', class_ = 'lister-item-year text-muted unbold').text
first_year

'(2017)'

In [7]:
first_movie.strong

<strong>8.1</strong>

In [8]:
first_imdb = float(first_movie.strong.text)
first_imdb

8.1

In [9]:
first_mscore = first_movie.find('span', class_ = 'metascore favorable')
first_mscore = int(first_mscore.text)
print(first_mscore)

77


In [10]:
first_votes = first_movie.find('span', attrs = {'name':'nv'})
first_votes

<span data-value="546325" name="nv">546,325</span>

In [11]:
first_votes['data-value']

'546325'

In [12]:
first_votes = int(first_votes['data-value'])

In [13]:
first_votes

546325

In [44]:
description = first_movie.find('div', class_ = 'lister-item-content').findAll('p', class_ = 'text-muted')[1].text.lstrip()


In [45]:
description

'In a future where mutants are nearly extinct, an elderly and weary Logan leads a quiet life. But when Laura, a mutant child pursued by scientists, comes to him for help, he must get her to safety.'

In [46]:
eighth_movie_mscore = movie_containers[7].find('div', class_ = 'ratings-metascore')
type(eighth_movie_mscore)

bs4.element.Tag

In [47]:
# Lists to store the scraped data in
names = []
years = []
imdb_ratings = []
metascores = []
votes = []
descriptions = []
# Extract data from individual movie container
for container in movie_containers:
    # If the movie has Metascore, then extract:
    if container.find('div', class_ = 'ratings-metascore') is not None:
        # The name
        name = container.h3.a.text
        names.append(name)
        # The year
        year = container.h3.find('span', class_ = 'lister-item-year').text
        years.append(year)
        # The IMDB rating
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)
        # The Metascore
        m_score = container.find('span', class_ = 'metascore').text
        metascores.append(int(m_score))
        # The number of votes
        vote = container.find('span', attrs = {'name':'nv'})['data-value']
        votes.append(int(vote))
        # The descriptions
        descp = container.find('div', class_ = 'lister-item-content').findAll('p', class_ = 'text-muted')[1].text.lstrip()
        descriptions.append(descp)

In [49]:
import pandas as pd
test_df = pd.DataFrame({'movie': names,
                       'year': years,
                       'imdb': imdb_ratings,
                       'metascore': metascores,
                       'votes': votes,
                        'descriptions': descriptions
})
print(test_df.info())
test_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 6 columns):
movie           45 non-null object
year            45 non-null object
imdb            45 non-null float64
metascore       45 non-null int64
votes           45 non-null int64
descriptions    45 non-null object
dtypes: float64(1), int64(2), object(3)
memory usage: 2.2+ KB
None


,movie,year,imdb,metascore,votes,descriptions
0,Logan,(2017),8.1,77,546325,"In a future where mutants are nearly extinct, ..."
1,Wonder Woman,(2017),7.5,76,476171,When a pilot crashes and tells of conflict in ...
2,Guardians of the Galaxy Vol. 2,(2017),7.7,67,461037,The Guardians struggle to keep together as a t...
3,Dunkirk,(2017),7.9,94,457189,"Allied soldiers from Belgium, the British Empi..."
4,Star Wars: Episode VIII - The Last Jedi,(2017),7.2,85,452703,Rey develops her newly discovered abilities wi...
5,Thor: Ragnarok,(2017),7.9,74,452463,"Thor is imprisoned on the planet Sakaar, and m..."
6,Spider-Man: Homecoming,(2017),7.5,73,405002,Peter Parker balances his life as an ordinary ...
7,Get Out,(I) (2017),7.7,84,386203,A young African-American visits his white girl...
8,Blade Runner 2049,(2017),8.0,81,369307,A young blade runner's discovery of a long-bur...
9,Baby Driver,(2017),7.6,86,353724,After being coerced into working for a crime b...


In [50]:
headers = {"Accept-Language": "en-US, en;q=0.5"}

In [51]:
pages = [str(i) for i in range(1,5)]
years_url = [str(i) for i in range(2000,2018)]

In [52]:
from time import sleep
from random import randint

In [53]:
from time import time
start_time = time()
requests = 0
for _ in range(5):
    # A request would go here
    requests += 1
    sleep(randint(1,3))
    elapsed_time = time() - start_time
    print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))

Request: 1; Frequency: 0.9963117284280533 requests/s
Request: 2; Frequency: 0.49886034017097514 requests/s
Request: 3; Frequency: 0.42768343577232765 requests/s
Request: 4; Frequency: 0.4434458531751895 requests/s
Request: 5; Frequency: 0.41579485609623695 requests/s


In [54]:
from IPython.core.display import clear_output
start_time = time()
requests = 0
for _ in range(5):
    # A request would go here
    requests += 1
    sleep(randint(1,3))
    current_time = time()
    elapsed_time = current_time - start_time
    print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

Request: 5; Frequency: 0.7106142796552121 requests/s


In [55]:
from warnings import warn
warn("Warning Simulation")

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Warning Simulation
  


In [56]:
# Redeclaring the lists to store data in
names = []
years = []
imdb_ratings = []
metascores = []
votes = []
descriptions = []
# Preparing the monitoring of the loop
start_time = time()
requests = 0

# For every year in the interval 2000-2017
for year_url in years_url:

    # For every page in the interval 1-4
    for page in pages:

        # Make a get request
        response = get('http://www.imdb.com/search/title?release_date=' + year_url +
        '&sort=num_votes,desc&page=' + page, headers = headers)

        # Pause the loop
        sleep(randint(8,15))

        # Monitor the requests
        requests += 1
        elapsed_time = time() - start_time
        print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
        clear_output(wait = True)

        # Throw a warning for non-200 status codes
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, response.status_code))

        # Break the loop if the number of requests is greater than expected
        if requests > 72:
            warn('Number of requests was greater than expected.')
            break

        # Parse the content of the request with BeautifulSoup
        page_html = BeautifulSoup(response.text, 'html.parser')

        # Select all the 50 movie containers from a single page
        mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')

        # For every movie of these 50
        for container in mv_containers:
            # If the movie has a Metascore, then:
            if container.find('div', class_ = 'ratings-metascore') is not None:

                # Scrape the name
                name = container.h3.a.text
                names.append(name)

                # Scrape the year
                year = container.h3.find('span', class_ = 'lister-item-year').text
                years.append(year)

                # Scrape the IMDB rating
                imdb = float(container.strong.text)
                imdb_ratings.append(imdb)

                # Scrape the Metascore
                m_score = container.find('span', class_ = 'metascore').text
                metascores.append(int(m_score))

                # Scrape the number of votes
                vote = container.find('span', attrs = {'name':'nv'})['data-value']
                votes.append(int(vote))
                
                # Scrape the descriptions
                descp = container.find('div', class_ = 'lister-item-content').findAll('p', class_ = 'text-muted')[1].text.lstrip()
                descriptions.append(descp)

Request:72; Frequency: 0.07522643234906264 requests/s


In [57]:
movie_ratings = pd.DataFrame({'movie': names,
                              'year': years,
                              'imdb': imdb_ratings,
                              'metascore': metascores,
                              'votes': votes,
                              'descriptions': descriptions
})
print(movie_ratings.info())
movie_ratings.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3296 entries, 0 to 3295
Data columns (total 6 columns):
movie           3296 non-null object
year            3296 non-null object
imdb            3296 non-null float64
metascore       3296 non-null int64
votes           3296 non-null int64
descriptions    3296 non-null object
dtypes: float64(1), int64(2), object(3)
memory usage: 154.6+ KB
None


,movie,year,imdb,metascore,votes,descriptions
0,Gladiator,(2000),8.5,67,1198157,A former Roman General sets out to exact venge...
1,Memento,(2000),8.5,80,1021383,A man with short-term memory loss attempts to ...
2,Snatch,(2000),8.3,55,711717,"Unscrupulous boxing promoters, violent bookmak..."
3,Requiem for a Dream,(2000),8.3,68,693282,The drug-induced utopias of four Coney Island ...
4,X-Men,(2000),7.4,64,530682,In a world where mutants (evolved super-powere...
5,Cast Away,(2000),7.8,73,468853,A FedEx executive undergoes a physical and emo...
6,American Psycho,(2000),7.6,64,426612,A wealthy New York City investment banking exe...
7,Unbreakable,(2000),7.3,62,343308,A man learns something extraordinary about him...
8,Meet the Parents,(2000),7.0,73,291666,Male nurse Greg Focker meets his girlfriend's ...
9,Mission: Impossible II,(2000),6.1,59,288655,"IMF Agent Ethan Hunt is sent to Sydney, to fin..."


In [58]:
movie_ratings = pd.DataFrame({'movie': names,
                              'year': years,
                              'imdb': imdb_ratings,
                              'metascore': metascores,
                              'votes': votes,
                              'descriptions': descriptions
})
print(movie_ratings.info())
movie_ratings.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3296 entries, 0 to 3295
Data columns (total 6 columns):
movie           3296 non-null object
year            3296 non-null object
imdb            3296 non-null float64
metascore       3296 non-null int64
votes           3296 non-null int64
descriptions    3296 non-null object
dtypes: float64(1), int64(2), object(3)
memory usage: 154.6+ KB
None


,movie,year,imdb,metascore,votes,descriptions
0,Gladiator,(2000),8.5,67,1198157,A former Roman General sets out to exact venge...
1,Memento,(2000),8.5,80,1021383,A man with short-term memory loss attempts to ...
2,Snatch,(2000),8.3,55,711717,"Unscrupulous boxing promoters, violent bookmak..."
3,Requiem for a Dream,(2000),8.3,68,693282,The drug-induced utopias of four Coney Island ...
4,X-Men,(2000),7.4,64,530682,In a world where mutants (evolved super-powere...
5,Cast Away,(2000),7.8,73,468853,A FedEx executive undergoes a physical and emo...
6,American Psycho,(2000),7.6,64,426612,A wealthy New York City investment banking exe...
7,Unbreakable,(2000),7.3,62,343308,A man learns something extraordinary about him...
8,Meet the Parents,(2000),7.0,73,291666,Male nurse Greg Focker meets his girlfriend's ...
9,Mission: Impossible II,(2000),6.1,59,288655,"IMF Agent Ethan Hunt is sent to Sydney, to fin..."


In [59]:
movie_ratings = movie_ratings[['movie', 'year', 'imdb', 'metascore', 'votes', 'descriptions']]
movie_ratings.head()

,movie,year,imdb,metascore,votes,descriptions
0,Gladiator,(2000),8.5,67,1198157,A former Roman General sets out to exact venge...
1,Memento,(2000),8.5,80,1021383,A man with short-term memory loss attempts to ...
2,Snatch,(2000),8.3,55,711717,"Unscrupulous boxing promoters, violent bookmak..."
3,Requiem for a Dream,(2000),8.3,68,693282,The drug-induced utopias of four Coney Island ...
4,X-Men,(2000),7.4,64,530682,In a world where mutants (evolved super-powere...


In [60]:
movie_ratings['year'].unique()

array(['(2000)', '(I) (2000)', '(2001)', '(2002)', '(2003)', '(2004)',
       '(I) (2004)', '(2005)', '(I) (2005)', '(2006)', '(I) (2006)',
       '(2007)', '(I) (2007)', '(2008)', '(I) (2008)', '(2009)',
       '(I) (2009)', '(2010)', '(I) (2010)', '(2011)', '(I) (2011)',
       '(2012)', '(I) (2012)', '(2013)', '(I) (2013)', '(2014)',
       '(I) (2014)', '(II) (2014)', '(2015)', '(I) (2015)', '(II) (2015)',
       '(2016)', '(II) (2016)', '(I) (2016)', '(IX) (2016)', '(2017)',
       '(I) (2017)'], dtype=object)

In [61]:
movie_ratings.loc[:, 'year'] = movie_ratings['year'].str[-5:-1].astype(int)

In [62]:
movie_ratings['year'].head(3)

0    2000
1    2000
2    2000
Name: year, dtype: int64

In [63]:
movie_ratings.describe().loc[['min', 'max'], ['imdb', 'metascore']]

,imdb,metascore
min,4.1,24.0
max,9.0,100.0


In [64]:
movie_ratings['n_imdb'] = movie_ratings['imdb'] * 10
movie_ratings.head(3)

,movie,year,imdb,metascore,votes,descriptions,n_imdb
0,Gladiator,2000,8.5,67,1198157,A former Roman General sets out to exact venge...,85.0
1,Memento,2000,8.5,80,1021383,A man with short-term memory loss attempts to ...,85.0
2,Snatch,2000,8.3,55,711717,"Unscrupulous boxing promoters, violent bookmak...",83.0


In [65]:
movie_ratings.to_csv('movie_ratings.csv')